In [835]:
####### USER INPUT #######

OpenBal = 29050    # type bal as at 1 Jan 2021
mydate = pd.to_datetime('2021-01-31')    # enter the date you want to forecast

In [836]:

def cat(mytext):
    
    # list of keywords, add as you find new ones
    WolliRent        = ['slick','carena','wolli income']
    WolliInterest    = ['LN REPAY 778047926','wolli expense','package annual']
    ThePondRent      = ['harcourts','hills Prestige','hillside','the pond income']
    ThePondInterest  = ['876891821','778047934','the pond expense','admiral']
    Grocery          = ['Coles','woolworths','woolworth','tongli','WW Metro','supermarket','iga']
    Entertainment    = ['rebel','chemist','mcdonald','malatang','kmart','office','chatime','kogan','tea','indo'
                       ,'pondok','bread','subway','gong','chinese','kfc','mie','ciak','japanese','thai',
                       'saigon','bakery','racha','taste','food','card pocket money','sushi','domino','coffee',
                       'denmar','kurtosh','toastie','mazesoba','dopa','yummy','satang','daiso','bunning',
                       'reject shop','dough','kitkat','harper','woollahra ps','cafe','marigold','decathlon',
                       '7-eleven','merlin','golden summit','lamb','the growers','pancake','petok','pendopo',
                       'makan','grandbuy','churro','harry','big w','jim','martabak','emperors','pork','juice'
                       ,'chai','kopi','target','mart','butcher','sawah','water','aqua','sheng','crocodile'
                       ,'mumbai','indian','nandos']
    Salary           = ['salary']
    Utilities        = ['TPG','ENERGY','AGL','Netflix','OPTUS','Gas','Electricity','Telco','Prime']
    Insurance        = ['life','ins','medibank']
    Car              = ['Top One','AAMI','BPme','Etoll','Petrol']
    Rental           = ['Quay','DEFT RENT']
    Excluded         = ['Payment','2k']
    Discretionary    = ['Transfer To','Cardless Cash','Reid Cycle','Ikea',
                        'Good Guys','Harvey','JB Hi-Fi','CrazySales','cosmetic','baby','zara','H&M'
                        ,'Cotton On','uniqlo']
    # combine all the categories into one list called collection
    collection      = [WolliRent,WolliInterest,ThePondRent,ThePondInterest,Grocery,Entertainment,Salary,
                       Utilities,Insurance,Car,Rental,Excluded,Discretionary]


    for i in range(len(collection)):
        for k in range(len(collection[i])):
            if collection[i][k].lower() in mytext.lower():
                if i == 0:
                    cat = 'Wolli - Income'
                elif i == 1:
                    cat = 'Wolli - Expense'
                elif i == 2:
                    cat = 'The Pond - Income'
                elif i == 3:
                    cat = 'The Pond - Expense'
                elif i == 4:
                    cat = 'Grocery'
                elif i == 5:
                    cat = 'Food & Entertainment'
                elif i == 6:
                    cat = 'Salary'
                elif i == 7:
                    cat = 'Utilities'   
                elif i == 8:
                    cat = 'Insurance'   
                elif i == 9:
                    cat = 'Car'   
                elif i == 10:
                    cat = 'Rental'
                elif i == 11:
                    cat = 'Exclude'
                elif i == 12:
                    cat = 'Discretionary'   
                return cat
            
   

In [837]:
import pandas as pd
import numpy as np

EndDate = '2021-12-31'


######## input your forecast/budget number to the list below ###########
     
            # Desc         Amt  Start Date  Freq Offfset
mylist = [['The Pond Income  - Rent'       ,650,'2021-01-5','','7D',0],
          ['The Pond Expense - WBC'        ,-889,'2021-01-6','','7D',0],
          ['The Pond Expense - CBA1'       ,-161,'2020-12-7','','MS',6],
          ['The Pond Expense - CBA2'       ,-307,'2020-12-9','','MS',9],
          ['The Pond Income  - Tree'       ,-400,'2021-01-15','2021-01-15','7D',0],    # adhoc just put end end date
          ['The Pond Income  - Water Pump' ,-1450,'2021-01-15','2021-01-15','7D',0],    # adhoc just put end e
          ['The Pond Income  - Water'      ,-147,'2021-01-1','','3MS',0],
          ['The Pond Income  - Council'    ,-400,'2021-02-28','','3MS',27],
          
          
          ['Wolli Income  - Rent'      ,450,'2021-02-06','','7D',0],
          ['Wolli Expense - Strata'    ,-1183,'2021-03-26','','3MS',25],
          ['Wolli Expense - Water'     ,-144,'2021-04-20','','3MS',19],
          ['Wolli Expense - Council'   ,-348,'2021-02-28','','3MS',27],
          ['Wolli Expense - CBA'       ,-311,'2021-01-6','','7D',0],
          ['Wolli Expense - Insurance' ,-35.59,'2021-01-15','','MS',15],
          
          
          ['Salary'              ,3413.17,'2021-01-13','','14D',0],
          ['Tax Return'          ,7000,'2021-07-30','2021-07-30','7D',0],
          ['Rent Quay St'        ,-700,'2021-01-4','','7D',0],
          ['Parents'             ,-100,'2021-01-4','','7D',0],
          ['Var Food & Ent'      ,-375,'2021-01-4','','7D',0],
          
          ['AAMI Car'            ,-94.2,'2020-12-18','','MS',17],
          ['Car Petrol'          ,-30,'2021-01-01','','7D',0],
          ['TPG'                 ,-59.99,'2020-12-2','','MS',1],
          ['Gas'                 ,-191,'2020-12-5','','MS',4],
          ['Electricity'         ,-238,'2021-02-25','','3MS',24],  #MS means beg of month, M means end of month
          ['Optus'               ,-60,'2020-12-21','','MS',20],
         ]

########################################################################


df = pd.DataFrame(columns=['Date','Description','Amount'])

for i in range(len(mylist)):
    if mylist[i][3] == '':
        ToDate = EndDate
        #print(ToDate,mylist[i][0])
    else:
        ToDate = mylist[i][3]
        #print(ToDate,mylist[i][0])
    date = pd.date_range(mylist[i][2],ToDate,freq=mylist[i][4]) + pd.DateOffset(days=mylist[i][5])
    amount = mylist[i][1]
    desc = mylist[i][0]
    tmp = pd.DataFrame({'Date':date,'Description':desc,'Amount':amount})
    df = df.append(tmp)   
    
# add Period
df['Period'] = pd.to_datetime(df['Date'],dayfirst=True).dt.to_period('M')
df['Cat']= df.apply(lambda x:cat(x['Description']),axis=1)
#df['YTD'] = df.groupby(['Description'])['Amount'].transform(pd.Series.cumsum)


# create forecast table for specific period
tbl_forecast = df[df['Date']<=mydate].sort_values('Cat')
forecast = tbl_forecast[['Cat','Amount']].groupby(['Cat']).sum().reset_index()


########### download and run actual from CBA statements etc   ##########


import pandas as pd
cba = pd.read_csv(r'/Users/ronny/Downloads/CSVData-9.csv', header=None,usecols=[0,1,2])
cba.columns=['Date','Amount','Description']


cba['Cat'] = cba.apply(lambda x:cat(x['Description']),axis=1)
cba['Cat'].fillna('Other',inplace=True)
# add Period
cba['Period'] = pd.to_datetime(cba['Date'],dayfirst=True).dt.to_period('M')
tbl_actual = cba

#cba.to_csv(r'C:\Users\m118954\mydsbook\Data\CSVData2.csv')

actual = cba.groupby('Cat').sum().reset_index()


budget = pd.merge(forecast,actual,on=['Cat'],how='left')
budget['Var'] = budget['Amount_y'] - budget['Amount_x']
budget.columns=['Category','Forecast','Actual','Variance']



In [838]:
pd.options.display.float_format = '{:,.2f}'.format
from IPython.display import display

import datetime 


bal = df[df['Date']<=mydate]['Amount'].sum()
df1 = df[df['Date']<=mydate].groupby(['Description']).sum()

display ('Your Forecast Income at the end of ' + str(mydate) + ' is ' + str(bal))
display ('Your Forecast Balance at the end of ' + str(mydate) + ' is ' + str(bal+OpenBal))


df1 = df1.reset_index()
df1['Cat']= df1.apply(lambda x:cat(x['Description']),axis=1)
df1 = df1[['Cat','Description','Amount']].sort_values('Cat')



print ('### This is how you are tracking against forecast in your CBA Offset ####')
display(budget)


print ('\n\n #### Report 2 - Granular Forecast ####')
display(df1)

'Your Forecast Income at the end of 2021-01-31 00:00:00 is -3093.8499999999995'

'Your Forecast Balance at the end of 2021-01-31 00:00:00 is 25956.15'

### This is how you are tracking against forecast in your CBA Offset ####


,Category,Forecast,Actual,Variance
0,Car,-244.20,-94.20,150.00
1,Food & Entertainment,"-1,500.00","-1,402.22",97.78
2,Rental,"-2,800.00","-2,100.00",700.00
3,Salary,"6,826.34","3,156.80","-3,669.54"
4,The Pond - Expense,"-4,024.00","-3,138.52",885.48
5,The Pond - Income,603.00,611.50,8.50
6,Utilities,-310.99,-311.99,-1.00
7,Wolli - Expense,"-1,244.00",-933.00,311.00




 #### Report 2 - Granular Forecast ####


,Cat,Description,Amount
0,Car,AAMI Car,-94.20
1,Car,Car Petrol,-150.00
15,Food & Entertainment,Var Food & Ent,"-1,500.00"
5,Rental,Rent Quay St,"-2,800.00"
6,Salary,Salary,"6,826.34"
8,The Pond - Expense,The Pond Expense - CBA1,-161.00
9,The Pond - Expense,The Pond Expense - CBA2,-307.00
10,The Pond - Expense,The Pond Expense - WBC,"-3,556.00"
11,The Pond - Income,The Pond Income - Rent,"2,600.00"
12,The Pond - Income,The Pond Income - Tree,-400.00


# Used Below query to see details of discrepancy ac vs forecast

In [839]:
from IPython.display import display


check = 'Car'
chk_forecast = tbl_forecast[tbl_forecast['Cat']==check]
chk_actual =  tbl_actual[tbl_actual['Cat']==check]

print ('#### Forecast Details ####')
display (chk_forecast)

print ('#### Actual Details ####')
display (chk_actual)


#### Forecast Details ####


,Date,Description,Amount,Period,Cat
1,2021-01-08,Car Petrol,-30,2021-01,Car
4,2021-01-29,Car Petrol,-30,2021-01,Car
3,2021-01-22,Car Petrol,-30,2021-01,Car
2,2021-01-15,Car Petrol,-30,2021-01,Car
0,2021-01-18,AAMI Car,-94.20,2021-01,Car
0,2021-01-01,Car Petrol,-30,2021-01,Car


#### Actual Details ####


,Date,Amount,Description,Cat,Period
9,18/01/2021,-94.20,Transfer to xx2574 NetBank AAMI Car,Car,2021-01


In [844]:
import pandas as pd

#import pandas as pd
#import numpy as np
#import os
#import glob

# pick up all offset files
#os.chdir("/Users/ronny/Downloads")
#extension = 'csv'
#offset_file = [i for i in glob.glob('*.{}'.format(extension))]




card = pd.read_csv(r'/Users/ronny/Downloads/CSVData-11.csv', header=None,usecols=[0,1,2])
card.columns=['Date','Amount','Description']
card['Cat'] = card.apply(lambda x:cat(x['Description']),axis=1)
card['Cat'].fillna('Other',inplace=True)
card['Period'] = pd.to_datetime(card['Date'],dayfirst=True).dt.to_period('M')

In [841]:
pd.pivot_table(card, values='Amount', index=['Cat'],
                    columns=['Period'], aggfunc=np.sum)

Period,2020-10,2020-11,2020-12,2021-01
Cat,,,,
Car,-212.08,-277.73,"-1,585.56",-94.20
Discretionary,-343.34,-636.09,-537.00,-9.00
Exclude,"5,112.80","3,204.00","4,161.43","2,352.55"
Food & Entertainment,"-1,283.10",-610.81,-659.84,-784.35
Grocery,-576.16,-621.45,-606.41,-609.71
Insurance,-486.48,-719.06,-499.07,-456.13
Other,-60.70,-38.59,-27.84,-24.40
Utilities,-202.39,-381.01,-138.10,-324.40
Wolli - Expense,-395.00,nan,nan,nan


In [842]:
card[(card['Period']=='2020-11') & (card['Cat']=='Insurance')]

,Date,Amount,Description,Cat,Period
224,29/11/2020,-32.94,WESTPAC LIFE INS Barangaroo NSW,Insurance,2020-11
236,27/11/2020,-31.99,Baby Village Bondi JunctiNSW,Insurance,2020-11
237,27/11/2020,-99.00,Baby Village Bondi JunctiNSW,Insurance,2020-11
238,27/11/2020,-99.00,Baby Village Bondi JunctiNSW,Insurance,2020-11
260,23/11/2020,-121.72,GENERAL INS HOME SYDNEY,Insurance,2020-11
274,19/11/2020,-59.78,WESTPAC LIFE INS Barangaroo NSW,Insurance,2020-11
275,19/11/2020,-56.38,WESTPAC LIFE INS Barangaroo NSW,Insurance,2020-11
276,19/11/2020,-18.27,WESTPAC LIFE INS Barangaroo NSW,Insurance,2020-11
280,16/11/2020,-38.13,GENERAL INS HOME SYDNEY,Insurance,2020-11
293,12/11/2020,-161.85,MEDIBANK PHI DEBIT MELBOURNE,Insurance,2020-11


In [843]:
card[card['Cat']=='Other'].head(60)

,Date,Amount,Description,Cat,Period
30,17/01/2021,-4.90,VICS AUSTRALIA EASTWOOD NSW,Other,2021-01
62,10/01/2021,-10.00,Q AND Y GROUP PTY LTD HAYMARKET,Other,2021-01
79,07/01/2021,-9.50,UNISON DESIGNS PTY LTD SYDNEY,Other,2021-01
109,30/12/2020,-4.30,HAYMARKET NEW YEN YE HAYMARKET NSW,Other,2020-12
134,22/12/2020,-6.60,J&Q GROUP PTY LTD HAYMARKET NSW,Other,2020-12
146,21/12/2020,-8.95,SERAFIC PTY LIMITED CHATSWOOD,Other,2020-12
154,18/12/2020,-7.99,ALLY FASHION SYDNEY NSW,Other,2020-12
219,30/11/2020,-1.10,NESPRESSO AUSTRALIA AUS,Other,2020-11
233,27/11/2020,-20.00,GroupTogether Pty Ltd. Sydney,Other,2020-11
243,27/11/2020,-15.90,UNISON DESIGNS PTY LTD SYDNEY,Other,2020-11


In [845]:
card.head()

,Date,Amount,Description,Cat,Period
0,25/01/2021,375.00,"PAYMENT RECEIVED, THANK YOU",Exclude,2021-01
1,22/01/2021,-3.95,WW METRO 1716 HAYMARKET NS,Grocery,2021-01
2,22/01/2021,-7.40,WW METRO 1716 HAYMARKET NS,Grocery,2021-01
3,22/01/2021,-57.00,OPTUS BILLING AUTOPAY MACQUARIE PANSW,Utilities,2021-01
4,22/01/2021,-37.00,KMART 1079 CHATSWOOD AUS,Food & Entertainment,2021-01


In [909]:
word_list = card['Description'].tolist()
word_counter = {}

for word in word_list:
    if word in word_counter:
         word_counter[word] += 1
    else:
         word_counter[word] = 1

popular_words = sorted(word_counter, key = word_counter.get, reverse = True)

display ('top 3 popular words are ' + str(popular_words[:3]))


for i in popular_words:
    print (i + ' ' + str(word_counter.get(i)))

"top 3 popular words are ['WW METRO        1716     HAYMARKET   NS', 'PAYMENT RECEIVED, THANK YOU', 'TONGLI SUPERMARKET QPS   HAYMARKET']"

WW METRO        1716     HAYMARKET   NS 73
PAYMENT RECEIVED, THANK YOU 59
TONGLI SUPERMARKET QPS   HAYMARKET 22
WESTPAC LIFE INS         Barangaroo  NSW 15
THAI KEE IGA             HAYMARKET   NSW 11
COLES 0839               BROADWAY    AUS 10
CHEMIST WAREHOUSE        HAYMARKET   NSW 9
GENERAL INS HOME         SYDNEY 8
IKEA TEMPE               TEMPE       NSW 6
RACHA THANI PL           SYDNEY 6
KMART 1155               BROADWAY    AUS 5
GRANDBUY INVESTMENT      SYDNEY      NSW 5
PRESTIGE COSMETIC TATT   SYDNEY 5
BP Australia BPme        1300130027  VIC 5
OPTUS BILLING AUTOPAY    MACQUARIE PANSW 4
AAMI                     BRISBANE    QLD 4
YUMMY THAI STREET FO     SYDNEY      NSW 4
NETFLIX.COM              Melbourne   AUS 4
MEDIBANK PHI DEBIT       MELBOURNE 4
COLES EXPRESS 1561       WATERLOO    AUS 4
MCDONALDS DARLING QTR    SYDNEY 4
THE REJECT SHOP 2299     GLEBE       NSW 3
MIRACLE SUPERMARKETS     SYDNEY      NSW 3
SQ *ENJOY MIE            Haymarket   AUS 3
ENERGYAUSTRALIA PTY LT  